Model

In [1]:
import pandas as pd

df1 = pd.read_csv('Formulation-Indications.csv')
formulations_lst = list(df1['Name of Medicine'])

processed_list = []
original_list = list(df1['Main Indications'])
for item in original_list:
    # Remove spaces and newline characters, convert to lowercase
    processed_item = ''.join(item.split()).lower()
    processed_list.append(processed_item)


['arsha,agnimandya,udararoga,vibandha', 'sarvajvara,jirnajvara', 'kandu,tvakvikara,vibandha', 'balaroga,balakshaya,agnimandya,aruchi', 'hridroga,hriddrava,hrid-daurbalya,moha,murchha']


Symptom description dataset

In [2]:
symptoms = pd.read_csv('ayurvedic_symptoms_desc.csv')
symptoms['Symptom'] = symptoms['Symptom'].str.lower()

def symptoms_desc(symptom_name):
    row = symptoms[symptoms['Symptom'] == symptom_name.lower()]
#     print(row)
    if not row.empty:
        description = row.iloc[0]['Description']
        print(f'Description of "{symptom_name}": {description}')
    else:
        print(f'Symptom "{symptom_name}" not found in the DataFrame.')

def symptoms_lst_desc(user_symptoms):
    for item in user_symptoms:
#         print(item)
        symptoms_desc(item)

In [3]:
# List of lists of symptoms
list_of_symptoms = processed_list

# Flatten the list of lists and split the symptoms using commas and spaces
flat_symptoms = [symptom.replace(',', ' ').split() for symptoms in list_of_symptoms for symptom in symptoms.split(',')]

# Get unique symptoms as a list
unique_symptoms = list(set(symptom for sublist in flat_symptoms for symptom in sublist))

import difflib

# Your list of correct words (assuming you have a list called unique_symptoms)
correct_words = unique_symptoms

def correct_symptoms(symptoms):
    corrected_symptoms = []
    for symptom in symptoms:
        corrected_symptom = difflib.get_close_matches(symptom, correct_words, n=1, cutoff=0.6)
        if corrected_symptom:
            corrected_symptoms.append(corrected_symptom[0])
        else:
            corrected_symptoms.append(symptom)
    return corrected_symptoms


In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

data = {
    "Formulation": formulations_lst,
    "Symptoms": processed_list,
}

df = pd.DataFrame(data)

# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Transform the symptom text data into numerical features
X_tfidf = tfidf_vectorizer.fit_transform(df['Symptoms'])

# Create and train a classifier (e.g., Naive Bayes)
clf = MultinomialNB()
clf.fit(X_tfidf, df['Formulation'])

# Spelling Correction
user_input = input("Enter a list of symptoms separated by spaces: ")
input_symptoms = user_input.split()
new_symptoms = correct_symptoms(input_symptoms)
print(f"Did you mean: {', '.join(new_symptoms)}")

# Find Symptom Description
symptoms_lst_desc(new_symptoms)

# Predict Formulation 
new_symptoms_tfidf = tfidf_vectorizer.transform(new_symptoms)
predicted_label = clf.predict(new_symptoms_tfidf)
print(f"Predicted Formulation: {predicted_label[0]}")


Did you mean: jvara, kasa
Description of "jvara": Fever.
Description of "kasa": Cough.
Predicted Formulation: Punarnavadi Kashayam


#Save in pickle

In [5]:
import pickle
pickle.dump(clf, open('model.pkl', 'wb'))